# Campo de gravidade sobre a margem leste brasileira

Este script mostra o campo de gravidade predito pelo modelo global [eigen 6c4](http://icgem.gfz-potsdam.de/ICGEM/documents/Foerste-et-al-EIGEN-6C4.pdf) (Förste et al., 2014). Os coeficientes da expansão em hamônicos esféricos que descreve este modelo global podem ser baixados no site do [IGCEM](http://icgem.gfz-potsdam.de/ICGEM/), na página [Table of models](http://icgem.gfz-potsdam.de/ICGEM/modelstab.html).

* Förste C., Bruinsma S.L., Abrikosov O., Lemoine J.-M., Schaller T., Götze H.-J., Ebbing J., Marty J.C., Flechtner F., Balmino G., Biancale R., 2014, EIGEN-6C4 The latest combined global gravity field model including GOCE data up to degree and order 2190 of GFZ Potsdam and GRGS Toulouse, 5th GOCE User Workshop, Paris, 25-28 November 2014, url: http://icgem.gfz-potsdam.de/ICGEM/documents/Foerste-et-al-EIGEN-6C4.pdf

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from pyproj import Geod

In [2]:
import gc

In [3]:
import gamma

### Dados baixados no site do [IGCEM](http://icgem.gfz-potsdam.de/ICGEM/)

#### Topografia/batimetria predita pelo modelo ETOPO1

Nos dados usados aqui, considere que a altura Geoidal é nula em toda a área. Neste caso, as altitudes geométrica e ortométrica são iguais.

In [4]:
etopo = pd.read_csv(
    filepath_or_buffer='aula_5c/etopo1-18525.gdf.txt',
    delimiter='\s+',
    names=['longitude', 'latitude', 'topografia'],
    skiprows=29
)

In [5]:
etopo

,longitude,latitude,topografia
0,319.0,-6.0,763.0
1,319.1,-6.0,830.0
2,319.2,-6.0,588.0
3,319.3,-6.0,562.0
4,319.4,-6.0,517.0
...,...,...,...
14236,328.6,-20.0,-4332.0
14237,328.7,-20.0,-4437.0
14238,328.8,-20.0,-4397.0
14239,328.9,-20.0,-4502.0


#### Distúrbio de gravidade

In [6]:
eigen_H_disturbio = pd.read_csv(
    filepath_or_buffer='aula_5c/eigen-6c4-18527.gdf.txt',
    delimiter='\s+',
    names=['H', 'disturbio'],
    usecols=(2,3),
    skiprows=35
)

In [7]:
eigen_H_disturbio

,H,disturbio
0,763.0,26.727723
1,830.0,35.081159
2,588.0,10.658780
3,562.0,6.824346
4,517.0,0.483257
...,...,...
14236,0.0,-4.540188
14237,0.0,-5.326420
14238,0.0,-6.192298
14239,0.0,-7.684612


#### Gravidade

In [8]:
eigen_gravidade = pd.read_csv(
    filepath_or_buffer='aula_5c/eigen-6c4-18528.gdf.txt',
    delimiter='\s+',
    names=['gravidade'],
    usecols=(3,),
    skiprows=34
)

In [9]:
eigen_gravidade

,gravidade
0,977884.247772
1,977871.796040
2,977922.023642
3,977926.165638
4,977933.660376
...,...
14236,978633.792703
14237,978633.024479
14238,978632.174005
14239,978630.695687


#### Dados combinados

In [10]:
dados_combinados = pd.concat(
    objs=[etopo, eigen_H_disturbio, eigen_gravidade],
    axis=1,
    join='outer'
)

In [11]:
dados_combinados

,longitude,latitude,topografia,H,disturbio,gravidade
0,319.0,-6.0,763.0,763.0,26.727723,977884.247772
1,319.1,-6.0,830.0,830.0,35.081159,977871.796040
2,319.2,-6.0,588.0,588.0,10.658780,977922.023642
3,319.3,-6.0,562.0,562.0,6.824346,977926.165638
4,319.4,-6.0,517.0,517.0,0.483257,977933.660376
...,...,...,...,...,...,...
14236,328.6,-20.0,-4332.0,0.0,-4.540188,978633.792703
14237,328.7,-20.0,-4437.0,0.0,-5.326420,978633.024479
14238,328.8,-20.0,-4397.0,0.0,-6.192298,978632.174005
14239,328.9,-20.0,-4502.0,0.0,-7.684612,978630.695687


In [12]:
# Apaga variáveis desnecessárias
del etopo, eigen_H_disturbio, eigen_gravidade
gc.collect()

0

### Parâmetros do datum geodésico WGS84

In [13]:
a, f, GM, omega = gamma.WGS84()

### Gravidade normal

**Sobre a topografia**

In [14]:
dados_combinados['gamma'] = gamma.closedform(a, f, GM, omega, dados_combinados['latitude'], dados_combinados['H'])

**Sobre o elipsoide WGS84**

In [15]:
dados_combinados['gamma-0'] = gamma.closedform(a, f, GM, omega, dados_combinados['latitude'], 
                                                        np.zeros_like(dados_combinados['H']))

### Anomalias calculadas

**Constante Gravitacional** (ou [*Newtonian constant of gravitation*](http://physics.nist.gov/cgi-bin/cuu/Category?view=html&Universal.x=58&Universal.y=11)) em $\frac{m^{3}}{kg \, s^{2}}$

In [16]:
G = 6.67408e-11

**Correção Ar-livre**

In [17]:
dados_combinados['ca'] = -0.3086*dados_combinados['H']

**Correção de Bouguer (platô)**

Neste cálculo foi considerado uma densidade de $\rho = \rho_{cc} \: \frac{kg}{m^{3}}$ para a correção sobre os continentes (altitudes positivas) e uma densidade de $\rho = (\rho_{co} - \rho_{a})\: \frac{kg}{m^{3}}$ para a correção sobre os oceanos (altitudes negativas), em que $\rho_{cc} = 2670 \: \frac{kg}{m^{3}}$, $\rho_{co} = 2900 \: \frac{kg}{m^{3}}$ e $\rho_{a} = 1025 \: \frac{kg}{m^{3}}$ representam, respectivamente, as densidades da crosta continental, crosta oceânica e da água dos oceanos.

In [18]:
rho_cc = 2670.
rho_co = 2900.
rho_a = 1025.

In [20]:
cb_continente = np.zeros(dados_combinados.shape[0], dtype=float)
cb_continente[dados_combinados['H'] > 0] = 2.*np.pi*G*1.0e5*rho_cc*dados_combinados['H'][dados_combinados['H'] > 0]

In [21]:
cb_oceano = np.zeros(dados_combinados.shape[0], dtype=float)
cb_oceano[dados_combinados['H'] < 0] = 2.*np.pi*G*1.0e5*(rho_co - rho_a)*dados_combinados['H'][dados_combinados['H'] < 0]

In [22]:
dados_combinados['cb'] = cb_continente + cb_oceano

In [23]:
del cb_continente, cb_oceano
gc.collect()

685

**Anomalia Ar-livre**

In [26]:
dados_combinados['ar-livre'] = (
    dados_combinados['gravidade'] - 
    dados_combinados['gamma-0'] - 
    dados_combinados['ca']
)

**Anomalia Bouguer (incompleta)**

In [27]:
dados_combinados['Bouguer'] = (
    dados_combinados['gravidade'] - 
    dados_combinados['gamma-0'] - 
    dados_combinados['ca'] - 
    dados_combinados['cb']
)

**Distúrbio de gravidade**

In [28]:
disturbio_calculado = dados_combinados['gravidade'] - dados_combinados['gamma']

**Moho isostática (modelo de Airy-Heiskanen)**

O modelo de Airy Heiskanen é um modelo de compensação isostática local, que considera a existência de um espessamento crustal $t_{c}$ sob os continentes e um afinamento crustal $t_{o}$ sob os oceanos. Geralmente, o espessamento crustal é denominado *raiz* e o afinamento crustal é denominado *antirraiz*. Ambos são definidos em relação a uma espessura crustal de referência $T$ e calculados em função da altitude $h$, das densidades $\rho_{cc}$ e $\rho_{co}$ das crostas continental e oceânica, respectivamente, da densidade $\rho_{m}$ do manto e da densidade $\rho_{a}$ da água dos oceanos.

$t_{c} = \dfrac{\rho_{cc}}{\rho_{m} - \rho_{cc}} \, h$, em que $h > 0$ (espessamento sob os continentes)

$t_{o} = \dfrac{\rho_{co} - \rho_{a}}{\rho_{m} - \rho_{co}} \, h$, em que $h < 0$ (afinamento sob os oceanos)

In [29]:
T = 40000.0 # metros
rho_m = 3270.0 

In [30]:
tc = np.zeros(dados_combinados.shape[0], dtype=float)
tc[dados_combinados['H'] > 0] = rho_cc*dados_combinados['H'][dados_combinados['H'] > 0]/(rho_m - rho_cc)

In [31]:
to = np.zeros(dados_combinados.shape[0], dtype=float)
to[dados_combinados['H'] < 0] = (rho_co - rho_a)*dados_combinados['H'][dados_combinados['H'] < 0]/(rho_m - rho_co)

In [32]:
dados_combinados['moho-AH'] = T + tc + to

In [33]:
del tc, to
gc.collect()

2407

**Correção isostática (Modelo de Airy-Heiskanen com aproximação do platô de Bouguer)**

A correção isostática consiste em remover o efeito gravitacional produzido por ondulações na base da crosta. A correção isostática utilizada neste trabalho é baseada no modelo de Airy-Heiskanen com aproximação do platô de Bouguer. Tal correção tem a seguinte expressão:

$ci_{c} = -2 \pi G \, \rho_{cc} h$, em que $h > 0$ (dados sobre os continentes)

$ci_{o} = 2 \pi G \, \left( \rho_{co} - \rho_{a} \right) h$, em que $h < 0$ (dados sobre o oceano)

In [34]:
cic = np.zeros(dados_combinados.shape[0], dtype=float)
cic[dados_combinados['H'] > 0] = 2.*np.pi*G*rho_cc*1.0e5*dados_combinados['H'][dados_combinados['H'] > 0]

In [36]:
cio = np.zeros(dados_combinados.shape[0], dtype=float)
cio[dados_combinados['H'] < 0] = -2.*np.pi*G*(rho_co - rho_a)*1.0e5*dados_combinados['H'][dados_combinados['H'] < 0]

In [37]:
dados_combinados['ci'] = cic + cio

In [38]:
dados_combinados['an-isostatica'] = dados_combinados['Bouguer'] - dados_combinados['ci']

In [39]:
del cic, cio
gc.collect()

1044

**Pontos iniciais e finais dos perfis analisados**

longitude_tmp, latitude_tmp = np.loadtxt('Linhas_sismicas/0066-0100_lon_lat', usecols=(0,1), unpack=True)
longitudes_perfis = [np.min(longitude_tmp), np.max(longitude_tmp)]
latitudes_perfis = [np.min(latitude_tmp), np.max(latitude_tmp)]

longitude_tmp, latitude_tmp = np.loadtxt('Linhas_sismicas/0259-1401_lon_lat', usecols=(0,1), unpack=True)
longitudes_perfis += [np.min(longitude_tmp), np.max(longitude_tmp)]
latitudes_perfis += [np.min(latitude_tmp), np.max(latitude_tmp)]

longitude_tmp, latitude_tmp = np.loadtxt('Linhas_sismicas/VB00-182_lon_lat', usecols=(0,1), unpack=True)
longitudes_perfis += [np.min(longitude_tmp), np.max(longitude_tmp)]
latitudes_perfis += [np.min(latitude_tmp), np.max(latitude_tmp)]

In [ ]:
longitudes_perfis = [360. - 40., 360. - 36.,
                     360. - 38.5, 360. - 34.5,
                     360. - 38., 360. - 34.,
                     360. - 36., 360. - 32.]
latitudes_perfis = [-14., -14.,
                    -12., -14.,
                    -10., -12.,
                    -8.5,  -11.]

In [ ]:
longitudes_perfis = np.array(longitudes_perfis)
latitudes_perfis = np.array(latitudes_perfis)

In [ ]:
p1, p2 = m(longitudes_perfis, latitudes_perfis)
perfis = np.transpose([p1,p2])

**Perfil 1 (mais ao sul)**

In [ ]:
perfil_topografia1 = gridder.profile(x, y, topografia, perfis[0,:], perfis[1,:], 200)

In [ ]:
perfil_bouguer1 = gridder.profile(x, y, bouguer, perfis[0,:], perfis[1,:], 200)

In [ ]:
perfil_disturbio1 = gridder.profile(x, y, disturbio, perfis[0,:], perfis[1,:], 200)

In [ ]:
perfil_moho_ah1 = gridder.profile(x, y, -moho_ah, perfis[0,:], perfis[1,:], 200)

In [ ]:
perfil_an_isostatica1 = gridder.profile(x, y, an_isostatica, perfis[0,:], perfis[1,:], 200)

**Perfil 2**

In [ ]:
perfil_topografia2 = gridder.profile(x, y, topografia, perfis[2,:], perfis[3,:], 200)

In [ ]:
perfil_bouguer2 = gridder.profile(x, y, bouguer, perfis[2,:], perfis[3,:], 200)

In [ ]:
perfil_disturbio2 = gridder.profile(x, y, disturbio, perfis[2,:], perfis[3,:], 200)

In [ ]:
perfil_moho_ah2 = gridder.profile(x, y, -moho_ah, perfis[2,:], perfis[3,:], 200)

In [ ]:
perfil_an_isostatica2 = gridder.profile(x, y, an_isostatica, perfis[2,:], perfis[3,:], 200)

**Perfil 3**

In [ ]:
perfil_topografia3 = gridder.profile(x, y, topografia, perfis[4,:], perfis[5,:], 200)

In [ ]:
perfil_bouguer3 = gridder.profile(x, y, bouguer, perfis[4,:], perfis[5,:], 200)

In [ ]:
perfil_disturbio3 = gridder.profile(x, y, disturbio, perfis[4,:], perfis[5,:], 200)

In [ ]:
perfil_moho_ah3 = gridder.profile(x, y, -moho_ah, perfis[4,:], perfis[5,:], 200)

In [ ]:
perfil_an_isostatica3 = gridder.profile(x, y, an_isostatica, perfis[4,:], perfis[5,:], 200)

**Perfil 4 (mais ao norte)**

In [ ]:
perfil_topografia4 = gridder.profile(x, y, topografia, perfis[6,:], perfis[7,:], 200)

In [ ]:
perfil_bouguer4 = gridder.profile(x, y, bouguer, perfis[6,:], perfis[7,:], 200)

In [ ]:
perfil_disturbio4 = gridder.profile(x, y, disturbio, perfis[6,:], perfis[7,:], 200)

In [ ]:
perfil_moho_ah4 = gridder.profile(x, y, -moho_ah, perfis[6,:], perfis[7,:], 200)

In [ ]:
perfil_an_isostatica4 = gridder.profile(x, y, an_isostatica, perfis[6,:], perfis[7,:], 200)

### Estatísticas e "mapas"

In [ ]:
topografia_min, topografia_media, topografia_max, topografia_var = mf.estatistica(topografia, 'm')

In [ ]:
mf.plota_mapa(m, x, y, topografia, area, 'm', 'Topografia/batimetria - ETOPO1', 'gist_earth', 
              (18, 12), 2., perfis, estados=True, escala=True, eixos=True)

In [ ]:
disturbio_min, disturbio_medio, disturbio_max, disturbio_var = mf.estatistica(disturbio, 'mGal')

In [ ]:
mf.plota_mapa(m, x, y, disturbio, area, 'mGal', 'Disturbio de gravidade - modelo eigen 6c4', 'RdBu_r',
              (18, 12), 2., perfis, estados=True, escala=True, eixos=True)

In [ ]:
disturbio_calc_min, disturbio_calc_medio, disturbio_calc_max, disturbio_calc_var = mf.estatistica(disturbio_calculado, 'mGal')

In [ ]:
mf.plota_mapa(m, x, y, disturbio_calculado, area, 'mGal', 'Disturbio de gravidade - calculado', 'RdBu_r',
              (18, 12), 2., perfis, estados=True, escala=True, eixos=True)

In [ ]:
ar_livre_min, ar_livre_medio, ar_livre_max, ar_livre_var = mf.estatistica(ar_livre, 'mGal')

In [ ]:
mf.plota_mapa(m, x, y, ar_livre, area, 'mGal', 'Anomalia Ar-livre - calculada', 'RdBu_r',
              (18, 12), 2., perfis, estados=True, escala=True, eixos=True)

In [ ]:
bouguer_min, bouguer_medio, bouguer_max, bouguer_var = mf.estatistica(bouguer, 'mGal')

In [ ]:
mf.plota_mapa(m, x, y, bouguer, area, 'mGal', 'Anomalia Bouguer (simples) - calculada', 'RdBu_r',
              (18, 12), 2., perfis, estados=True, escala=True, eixos=True)

In [ ]:
moho_ah_min, moho_ah_media, moho_ah_max, moho_ah_var = mf.estatistica(moho_ah, 'm')

In [ ]:
mf.plota_mapa(m, x, y, moho_ah, area, 'm', 'Moho calculada (Airy-Heiskanen)', 'terrain',
              (18, 12), 2., perfis, estados=True, escala=True, eixos=True)

In [ ]:
an_isostatica_min, an_isostatica_media, an_isostatica_max, an_isostatica_var = mf.estatistica(an_isostatica, 'm')

In [ ]:
mf.plota_mapa(m, x, y, an_isostatica, area, 'm', 'Anomalia isostatica (Airy-Heiskanen + aproximacao plato Bouguer)', 'RdBu_r',
              (18, 12), 2., perfis, estados=True, escala=True, eixos=True)

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(14, 6))
ax1, ax2 = axes
d = perfil_topografia1[2]*0.001
ax2.fill_between([d.min(), d.max()], [0, 0], -6000, color='#2780E3')
ax2.fill_between([d.min(), d.max()], -6000, -45000, color='#31a354')
ax2.fill_between(d, perfil_topografia1[3], perfil_moho_ah1[3], color='#333333')
ax2.set_ylabel('Topografia (m)')
ax2.set_xlabel('Distancia (km)')
ax2.set_ylim(-45000., 5000.)

ax1.set_title('Perfil 1 - mais ao sul', fontsize=16)
ax1.set_ylabel('Disturbio e Anomalia Bouguer (mGal)')
ax1.plot(d, perfil_bouguer1[3], '-', label='Bouguer')
ax1.plot(d, perfil_disturbio1[3], '-', label='Disturbio')
ax1.plot(d, perfil_an_isostatica1[3], '-', label='An. isostatica')
ax1.hlines(0, d.min(), d.max(), linestyles='--', color='#333333')
ax1.legend(loc='center right')
ax1.set_xlim(d.min(), d.max())
plt.tight_layout(h_pad=0, w_pad=0, pad=0)
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(14, 6))
ax1, ax2 = axes
d = perfil_topografia2[2]*0.001
ax2.fill_between([d.min(), d.max()], [0, 0], -6000, color='#2780E3')
ax2.fill_between([d.min(), d.max()], -6000, -45000, color='#31a354')
ax2.fill_between(d, perfil_topografia2[3], perfil_moho_ah2[3], color='#333333')
ax2.set_ylabel('Topografia (m)')
ax2.set_xlabel('Distancia (km)')
ax2.set_ylim(-45000., 5000.)

ax1.set_title('Perfil 2', fontsize=16)
ax1.set_ylabel('Disturbio e Anomalia Bouguer (mGal)')
ax1.plot(d, perfil_bouguer2[3], '-', label='Bouguer')
ax1.plot(d, perfil_disturbio2[3], '-', label='Disturbio')
ax1.plot(d, perfil_an_isostatica2[3], '-', label='An. isostatica')
ax1.hlines(0, d.min(), d.max(), linestyles='--', color='#333333')
ax1.legend(loc='center right')
ax1.set_xlim(d.min(), d.max())
plt.tight_layout(h_pad=0, w_pad=0, pad=0)
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(14, 6))
ax1, ax2 = axes
d = perfil_topografia3[2]*0.001
ax2.fill_between([d.min(), d.max()], [0, 0], -6000, color='#2780E3')
ax2.fill_between([d.min(), d.max()], -6000, -45000, color='#31a354')
ax2.fill_between(d, perfil_topografia3[3], perfil_moho_ah3[3], color='#333333')
ax2.set_ylabel('Topografia (m)')
ax2.set_xlabel('Distancia (km)')
ax2.set_ylim(-45000., 5000.)

ax1.set_title('Perfil 3', fontsize=16)
ax1.set_ylabel('Disturbio e Anomalia Bouguer (mGal)')
ax1.plot(d, perfil_bouguer3[3], '-', label='Bouguer')
ax1.plot(d, perfil_disturbio3[3], '-', label='Disturbio')
ax1.plot(d, perfil_an_isostatica3[3], '-', label='An. isostatica')
ax1.hlines(0, d.min(), d.max(), linestyles='--', color='#333333')
ax1.legend(loc='center right')
ax1.set_xlim(d.min(), d.max())
plt.tight_layout(h_pad=0, w_pad=0, pad=0)
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(14, 6))
ax1, ax2 = axes
d = perfil_topografia4[2]*0.001
ax2.fill_between([d.min(), d.max()], [0, 0], -6000, color='#2780E3')
ax2.fill_between([d.min(), d.max()], -6000, -45000, color='#31a354')
ax2.fill_between(d, perfil_topografia4[3], perfil_moho_ah4[3], color='#333333')
ax1.hlines(0, d.min(), d.max(), linestyles='--', color='#333333')
ax2.set_ylabel('Topografia (m)')
ax2.set_xlabel('Distancia (km)')
ax2.set_ylim(-45000., 5000.)

ax1.set_title('Perfil 4 - mais ao norte', fontsize=16)
ax1.set_ylabel('Disturbio e Anomalia Bouguer (mGal)')
ax1.plot(d, perfil_bouguer4[3], '-', label='Bouguer')
ax1.plot(d, perfil_disturbio4[3], '-', label='Disturbio')
ax1.plot(d, perfil_an_isostatica4[3], '-', label='An. isostatica')
ax1.hlines(0, d.min(), d.max(), linestyles='--', color='#333333')
ax1.legend(loc='center right')
ax1.set_xlim(d.min(), d.max())
plt.tight_layout(h_pad=0, w_pad=0, pad=0)
plt.show()

In [ ]:
diferenca1 = perfil_an_isostatica1[3] - perfil_disturbio1[3]
diferenca2 = perfil_an_isostatica2[3] - perfil_disturbio2[3]
diferenca3 = perfil_an_isostatica3[3] - perfil_disturbio3[3]
diferenca4 = perfil_an_isostatica4[3] - perfil_disturbio4[3]

In [ ]:
plt.figure(figsize=(10, 4))
plt.ylabel('Anomalia isostatica - Disturbio')
plt.plot(d, diferenca1, '-', label='Perfil 1')
plt.plot(d, diferenca2, '-', label='Perfil 2')
plt.plot(d, diferenca3, '-', label='Perfil 3')
plt.plot(d, diferenca4, '-', label='Perfil 4')
plt.hlines(0.0, d.min(), d.max(), linestyles='--', color='k')
plt.legend(loc='best')
plt.xlim(d.min(), d.max())
plt.tight_layout(h_pad=0, w_pad=0, pad=0)
plt.show()

Todos os perfis mostrados acima começam no continente e terminam no oceano. Os perfis 1, 2, 3 e 4 estão dispostos, respectivamente, de sul para norte.